In [ ]:
        strings=["2body_lep","2body_tau","3body_H","3body_tau"]
        for l in range(len(strings)):
            if br_file_alec[0:len(strings[l])]==strings[l]:
                new_file=strings[l]
                for m in range(len(br_file_alec)-3):
                    if br_file_alec[m:m+3].isdigit():
                        #pid_new=str(-abs(int(br_file_alec[m:m+3])))
                        #new_file+=pid_new
                        if br_file_alec[m-1:m]=="-":
                            pid_new=str(-int(br_file_alec[m-1:m+3]))
                            new_file+="_"+pid_new
                        else:
                            pid_new=str(-int(br_file_alec[m:m+3]))
                            new_file+="_"+pid_new
                    if br_file_alec[m:m+2].isdigit() and br_file_alec[m:m+3].isdigit()==False and br_file_alec[m-1:m+2].isdigit()==False:
                        if br_file_alec[m-1:m]=="-":
                            pid_new=str(-int(br_file_alec[m-1:m+2]))
                            new_file+="_"+pid_new
                        else:
                            pid_new=str(-int(br_file_alec[m:m+2]))
                            new_file+="_"+pid_new
                new_file+=".txt"
        br_file_alec=new_file
        print("conjugate file, ",br_file_alec)
        br_files_alec.remove(br_file_alec)